#Regressao usando base de Carros usados com Validação Cruzada

In [ ]:
#Importar dados essenciais
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score

!pip install scikeras

from scikeras.wrappers import KerasRegressor


from google.colab import drive
drive.mount('/content/drive/')

dataset = pd.read_csv('/content/drive/My Drive/CursoIA/RegressaoCarrosUsados/autos.csv', encoding='ISO-8859-1')

#Removendo colunas de dataset que não tem relação com o preço do Veículo

dataset = dataset.drop('dateCrawled', axis=1)
dataset = dataset.drop('dateCreated', axis=1)
dataset = dataset.drop('postalCode', axis=1)
dataset = dataset.drop('nrOfPictures', axis=1)
dataset = dataset.drop('lastSeen', axis=1)
dataset = dataset.drop('name', axis=1)
dataset = dataset.drop('seller', axis=1)
dataset = dataset.drop('offerType', axis=1)
inconsistent = dataset.loc[dataset.price <=10]
dataset = dataset.loc[dataset.price>10]
inconsistent = dataset.loc[dataset.price>350000]
dataset = dataset.loc[dataset.price<350000]

#Removendo registros NAN
print(dataset['vehicleType'].value_counts()) #limousine
print(dataset['gearbox'].value_counts()) #manuell
print(dataset['model'].value_counts()) #golf
print(dataset['fuelType'].value_counts()) #benzin
print(dataset['notRepairedDamage'].value_counts()) #nein

values = {'vehicleType':'limousine',
          'gearbox':'manuell',
          'model':'golf',
          'fuelType':'benzin',
          'notRepairedDamage':'nein'}
dataset = dataset.fillna(value = values)

previsores = dataset.iloc[:, 1:13].values
preco_real = dataset.iloc[:, 0].values

labelEncoder_previsores =  LabelEncoder()

previsores[:,0] = labelEncoder_previsores.fit_transform(previsores[:,0])
previsores[:,1] = labelEncoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = labelEncoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = labelEncoder_previsores.fit_transform(previsores[:,5])
previsores[:,8] = labelEncoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = labelEncoder_previsores.fit_transform(previsores[:,9])
previsores[:,10] = labelEncoder_previsores.fit_transform(previsores[:,10])


columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0,1,3,5,8,9,10])],     remainder='passthrough')
previsores=columnTransformer.fit_transform(previsores).toarray()

# Hora de criar a rede neural

def criar_rede():
  regressor = Sequential()

  regressor.add(Dense(units = 158, activation = 'relu', input_dim = 316))
  regressor.add(Dense(units = 158, activation = 'relu'))
  regressor.add(Dense(units = 1, activation = 'linear'))
  regressor.compile(loss = 'mean_absolute_error',
                    optimizer = 'adam',
                    metrics = ['mean_absolute_error'])
  return regressor


Mounted at /content/drive/
limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64
manuell      266547
automatik     75508
Name: gearbox, dtype: int64
golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
serie_1                1
discovery_sport        1
Name: model, Length: 251, dtype: int64
benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64
nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64


#Realizando validação cruzada

In [ ]:
regressor = KerasRegressor(build_fn = criar_rede,
                           epochs = 100,
                           batch_size = 300)

resultados = cross_val_score(estimator = regressor,
                             X = previsores, y = preco_real,
                             cv = 10, scoring = 'neg_mean_absolute_error')
media = resultados.mean()
desvio = resultados.std()

print(media, desvio)

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 8s 7ms/step - loss: 3892.5659 - mean_absolute_error: 3892.5659
Epoch 2/100
1078/1078 [==============================] - 9s 8ms/step - loss: 3298.4341 - mean_absolute_error: 3298.4341
Epoch 3/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2974.9385 - mean_absolute_error: 2974.9385
Epoch 4/100
1078/1078 [==============================] - 9s 9ms/step - loss: 2869.6604 - mean_absolute_error: 2869.6604
Epoch 5/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2796.6521 - mean_absolute_error: 2796.6521
Epoch 6/100
1078/1078 [==============================] - 9s 9ms/step - loss: 2767.0242 - mean_absolute_error: 2767.0242
Epoch 7/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2725.5110 - mean_absolute_error: 2725.5110
Epoch 8/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2685.5281 - mean_absolute_error: 2685.5281
Epoch 9/100
1078/1078 [=================

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 10s 9ms/step - loss: 3794.7285 - mean_absolute_error: 3794.7285
Epoch 2/100
1078/1078 [==============================] - 6s 6ms/step - loss: 3295.9941 - mean_absolute_error: 3295.9941
Epoch 3/100
1078/1078 [==============================] - 9s 9ms/step - loss: 3015.1587 - mean_absolute_error: 3015.1587
Epoch 4/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2866.3137 - mean_absolute_error: 2866.3137
Epoch 5/100
1078/1078 [==============================] - 8s 8ms/step - loss: 2813.6580 - mean_absolute_error: 2813.6580
Epoch 6/100
1078/1078 [==============================] - 8s 7ms/step - loss: 2752.4673 - mean_absolute_error: 2752.4673
Epoch 7/100
1078/1078 [==============================] - 8s 7ms/step - loss: 2710.8394 - mean_absolute_error: 2710.8394
Epoch 8/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2669.7551 - mean_absolute_error: 2669.7551
Epoch 9/100
1078/1078 [================

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


...

Epoch 1/100
1078/1078 [==============================] - 9s 7ms/step - loss: 3891.8743 - mean_absolute_error: 3891.8743
Epoch 2/100
1078/1078 [==============================] - 8s 7ms/step - loss: 3387.9683 - mean_absolute_error: 3387.9683
Epoch 3/100
1078/1078 [==============================] - 10s 9ms/step - loss: 3091.6260 - mean_absolute_error: 3091.6260
Epoch 4/100
1078/1078 [==============================] - 10s 9ms/step - loss: 2895.9348 - mean_absolute_error: 2895.9346
Epoch 5/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2813.4470 - mean_absolute_error: 2813.4470
Epoch 6/100
1078/1078 [==============================] - 8s 8ms/step - loss: 2773.8513 - mean_absolute_error: 2773.8513
Epoch 7/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2723.5886 - mean_absolute_error: 2723.5884
Epoch 8/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2685.3992 - mean_absolute_error: 2685.3992
Epoch 9/100
1078/1078 [===============

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 10s 9ms/step - loss: 3853.8979 - mean_absolute_error: 3853.8979
Epoch 2/100
1078/1078 [==============================] - 7s 6ms/step - loss: 3313.2522 - mean_absolute_error: 3313.2522
Epoch 3/100
1078/1078 [==============================] - 8s 8ms/step - loss: 3020.3093 - mean_absolute_error: 3020.3093
Epoch 4/100
1078/1078 [==============================] - 8s 7ms/step - loss: 2868.6106 - mean_absolute_error: 2868.6104
Epoch 5/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2798.3508 - mean_absolute_error: 2798.3508
Epoch 6/100
1078/1078 [==============================] - 9s 9ms/step - loss: 2753.4705 - mean_absolute_error: 2753.4705
Epoch 7/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2725.7439 - mean_absolute_error: 2725.7441
Epoch 8/100
1078/1078 [==============================] - 10s 9ms/step - loss: 2668.0337 - mean_absolute_error: 2668.0337
Epoch 9/100
1078/1078 [===============

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
1078/1078 [==============================] - 10s 8ms/step - loss: 3859.0261 - mean_absolute_error: 3859.0261
Epoch 2/100
1078/1078 [==============================] - 7s 6ms/step - loss: 3298.2771 - mean_absolute_error: 3298.2771
Epoch 3/100
1078/1078 [==============================] - 10s 9ms/step - loss: 3047.6326 - mean_absolute_error: 3047.6326
Epoch 4/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2919.3469 - mean_absolute_error: 2919.3469
Epoch 5/100
1078/1078 [==============================] - 10s 9ms/step - loss: 2826.9253 - mean_absolute_error: 2826.9253
Epoch 6/100
1078/1078 [==============================] - 7s 6ms/step - loss: 2784.9343 - mean_absolute_error: 2784.9343
Epoch 7/100
1078/1078 [==============================] - 9s 8ms/step - loss: 2728.4436 - mean_absolute_error: 2728.4436
Epoch 8/100
1078/1078 [==============================] - 8s 7ms/step - loss: 2692.4343 - mean_absolute_error: 2692.4343
Epoch 9/100
1078/1078 [==============